In [1]:
import pyrqa
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('codes_scores_merged.csv')

In [3]:
a = data['Primary code '].unique()

In [4]:
a

array(['SMC', 'CM', 'CE', 'SSI', 'CRF', 'SESU', 'SN', 'CP', 'EM', 'TD'],
      dtype=object)

In [5]:
from numpy import asarray
from sklearn.preprocessing import LabelEncoder
# define data
a = data['Primary code ']
# define one hot encoding
encoder = LabelEncoder()
# transform data
encoder.fit(a)

LabelEncoder()

In [6]:
codes_list = data.groupby(['team_id'])['Primary code '].apply(list)

In [8]:
from pyrqa.analysis_type import Cross
from pyrqa.time_series import TimeSeries
from pyrqa.settings import Settings
from pyrqa.analysis_type import Classic
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation

x = []


for i in codes_list:
    data_points_x = encoder.transform(i)
    time_series_x = TimeSeries(data_points_x,
                               embedding_dimension=1,
                               time_delay=0)
    time_series = (time_series_x,
                   time_series_x)
    settings = Settings(time_series,
                        analysis_type=Cross,
                        neighbourhood=FixedRadius(0.5),
                        similarity_measure=EuclideanMetric,
                        theiler_corrector=0)
    computation = RQAComputation.create(settings,
                                        verbose=False)
    result = computation.run()
    result.min_diagonal_line_length = 2
    result.min_vertical_line_length = 2
    result.min_white_vertical_line_length = 2
    #print(result)
    x.append([result.determinism, result.recurrence_rate, result.entropy_diagonal_lines,
             result.longest_diagonal_line, result.laminarity,
             result.divergence, len(data_points_x)])
    

In [9]:
import numpy as np

Xtrain = pd.DataFrame(x)
#Xtrain = Xtrain.transpose()
Xtrain.columns = ["determinism", "recurrence_rate", "entropy", "longest_diagonal_line", "laminarity",
                 "divergence", "num_messages"]

Ytrain = data.groupby(['team_id'])['high_performance'].unique()

In [10]:
Xtrain['num_messages']

0      27
1      10
2      25
3      44
4      45
       ..
816    84
817    21
818    64
819    10
820    37
Name: num_messages, Length: 821, dtype: int64

In [11]:
# importing libraries
import statsmodels.api as sm

# building the model and fitting the data
log_reg = sm.Logit(np.asarray(Ytrain).astype(float), Xtrain.astype(float)).fit()


         Current function value: 0.297953
         Iterations: 35


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [12]:
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  821
Model:                          Logit   Df Residuals:                      814
Method:                           MLE   Df Model:                            6
Date:                Thu, 13 May 2021   Pseudo R-squ.:                  0.1252
Time:                        14:56:17   Log-Likelihood:                -244.62
converged:                      False   LL-Null:                       -279.63
Covariance Type:            nonrobust   LLR p-value:                 4.063e-13
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
determinism             -17.3767      4.509     -3.854      0.000     -26.213      -8.540
recurrence_rate          26.5851      7.582      3.506      0.000      11.724      41.446
entropy                   0.7676      1.442      0.532      0.595      -2.059       3.594
longest_diagonal_line    -0.0013   7.82e+04  -1.69e-08      1.000   -1.53e+05    1.53e+05
laminarity                2.2270      2.374      0.938      0.348      -2.426       6.880
divergence              -29.1490      7.583     -3.844      0.000     -44.011     -14.287
num_messages             -0.0015   7.82e+04  -1.96e-08      1.000   -1.53e+05    1.53e+05
=========================================================================================
"""